# PSTAT 194 Final Project: Predicting Flight Delays
## Team: Walk in the Spark
* Andrew Zhang
* Wendy Gao
* Alex Wu
* Shon Inouye

# Preprocessing

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

## Import dataset

In [2]:
flights_df = pd.read_csv("/mnt/data/flights.csv")

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
flights_df.index = pd.RangeIndex(len(flights_df.index))

In [4]:
flights_df.isnull().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER              14721
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
dtype: int64

In [5]:
flights_agg = flights_df[['MONTH','DAY','DAY_OF_WEEK','AIRLINE','ORIGIN_AIRPORT',
                          'SCHEDULED_DEPARTURE','SCHEDULED_TIME',
                          'DISTANCE','SCHEDULED_ARRIVAL','DEPARTURE_DELAY']].copy()
flights_agg = flights_agg.dropna(axis=0, how = "any")

In [6]:
flights_agg.isnull().sum()

MONTH                  0
DAY                    0
DAY_OF_WEEK            0
AIRLINE                0
ORIGIN_AIRPORT         0
SCHEDULED_DEPARTURE    0
SCHEDULED_TIME         0
DISTANCE               0
SCHEDULED_ARRIVAL      0
DEPARTURE_DELAY        0
dtype: int64

In [7]:
flights_agg['DELAY'] = 'NA'
flights_agg['DELAY'] = np.where(flights_agg['DEPARTURE_DELAY'] <= 0, 0, 1) #more efficient

## Data Balance

In [8]:
no_delay = (flights_agg['DELAY'] == 0).sum()
nobs = len(flights_agg['DELAY'])
no_delay_perc = float(no_delay)/nobs
delay_perc = 1 - no_delay_perc
print(no_delay_perc, delay_perc)

(0.6292264674895167, 0.3707735325104833)


## Undersampling

In [9]:
delay = nobs - no_delay
no_delay_indices = flights_agg[flights_agg.DELAY == 0].index
#undersamples no delays to equal same number of delays
np.random.seed(5)
random_indices = np.random.choice(no_delay_indices, delay, replace=False) 
no_delay_sample = flights_agg.loc[random_indices]

In [10]:
no_delay_sample[:10] #Check to make sure it was properly created

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_DELAY,DELAY
2483768,6,6,6,EV,DFW,2020,140.0,785,2340,-2.0,0
4298638,9,25,5,MQ,ORD,1142,73.0,177,1355,-2.0,0
1440480,4,3,5,WN,ATL,820,275.0,1587,955,-1.0,0
2073491,5,12,2,DL,DTW,1359,108.0,502,1547,-4.0,0
4999474,11,9,1,AS,BLI,640,49.0,93,729,-1.0,0
37497,1,3,6,DL,ATL,1225,102.0,515,1407,-4.0,0
38622,1,3,6,US,PHL,1340,107.0,449,1527,-4.0,0
4529081,10,9,5,WN,12889,2135,60.0,258,2235,-2.0,0
3452835,8,3,1,OO,MSP,1530,247.0,1535,1737,-4.0,0
2298058,5,26,2,EV,ATL,1449,124.0,692,1653,-5.0,0


#### Create balanced sample

In [11]:
delay_sample = flights_agg[flights_agg.DELAY == 1] 
flights_agg_balanced = delay_sample.append(no_delay_sample)

In [14]:
n = int(len(flights_agg_balanced)*0.10)
flights_new_bal = flights_agg_balanced.sample(n, random_state = 314)

## Double check to make sure new sample is balanced

In [15]:
no_delay_bal = (flights_new_bal['DELAY'] == 0).sum()
no_delay_perc_bal = float(no_delay_bal)/n
delay_perc_bal = 1 - no_delay_perc_bal
print(no_delay_perc_bal, delay_perc_bal)

(0.49846515008597514, 0.5015348499140249)


## Compare original dataset with new sample

In [16]:
flights_agg.describe()

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_DELAY,DELAY
count,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06
mean,6.547214e+00,1.570853e+01,3.932390e+00,1.328963e+03,1.419380e+02,8.247620e+02,1.493375e+03,9.370097e+00,3.707735e-01
std,3.397111e+00,8.774777e+00,1.986191e+00,4.834725e+02,7.533436e+01,6.087944e+02,5.068416e+02,3.708078e+01,4.830120e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.800000e+01,2.100000e+01,1.000000e+00,-8.200000e+01,0.000000e+00
25%,4.000000e+00,8.000000e+00,2.000000e+00,9.160000e+02,8.600000e+01,3.730000e+02,1.110000e+03,-5.000000e+00,0.000000e+00
50%,7.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.230000e+02,6.500000e+02,1.520000e+03,-2.000000e+00,0.000000e+00
75%,9.000000e+00,2.300000e+01,6.000000e+00,1.730000e+03,1.740000e+02,1.065000e+03,1.917000e+03,7.000000e+00,1.000000e+00
max,1.200000e+01,3.100000e+01,7.000000e+00,2.359000e+03,7.180000e+02,4.983000e+03,2.400000e+03,1.988000e+03,1.000000e+00


In [17]:
flights_new_bal.describe() #matches

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_DELAY,DELAY
count,425123.000000,425123.000000,425123.000000,425123.000000,425123.000000,425123.000000,425123.000000,425123.000000,425123.000000
mean,6.509756,15.695813,3.931145,1354.082113,143.492940,837.196708,1516.922794,14.237451,0.501535
std,3.400256,8.772097,1.983248,479.301012,76.069767,613.570598,507.189757,41.882479,0.499998
min,1.000000,1.000000,1.000000,1.000000,18.000000,31.000000,1.000000,-41.000000,0.000000
25%,4.000000,8.000000,2.000000,944.000000,87.000000,386.000000,1128.000000,-4.000000,0.000000
50%,7.000000,16.000000,4.000000,1345.000000,125.000000,668.000000,1544.000000,1.000000,1.000000
75%,9.000000,23.000000,6.000000,1740.000000,175.000000,1075.000000,1930.000000,14.000000,1.000000
max,12.000000,31.000000,7.000000,2359.000000,718.000000,4983.000000,2359.000000,1523.000000,1.000000


#### Build Airport dictionary to replace incomplete IATA airport codes for October

In [18]:
airport_table = pd.read_csv("/mnt/data/airport_codes_clean.csv")

In [19]:
oct_flights = flights_new_bal[flights_new_bal.MONTH == 10] 
flights_new_bal = flights_new_bal[~flights_new_bal["MONTH"].isin([10])]

In [20]:
oct_flights["ORIGIN_AIRPORT"] = oct_flights["ORIGIN_AIRPORT"].apply(pd.to_numeric)
oct_flights["ORIGIN_AIRPORT"].replace(list(airport_table['Code']), 
                                                                list(airport_table['Airport Code']), inplace=True)
flights_new_bal = flights_new_bal.append(oct_flights)

#### Give unique airport IATA code unique integer and for airlines

In [21]:
unique_IATA = flights_new_bal['ORIGIN_AIRPORT'].unique()
unique_IATA_id = [x for x in range(1, len(unique_IATA) + 1)]
flights_new_bal['ORIGIN_AIRPORT'].replace(unique_IATA, unique_IATA_id, inplace = True)

In [22]:
unique_airline = flights_new_bal['AIRLINE'].unique()
unique_airline_id = [x for x in range(1, len(unique_airline) + 1)]
flights_new_bal['AIRLINE'].replace(unique_airline, unique_airline_id, inplace = True)

## Initializing SQL Dataframe

In [24]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

## Save parquet file

#### Saving a parquet file will allow us to read the data directly into a pyspark sql dataframe

In [25]:
full_df = sqlCtx.createDataFrame(flights_new_bal)
full_df.show(5)

+-----+---+-----------+-------+--------------+-------------------+--------------+--------+-----------------+---------------+-----+
|MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|SCHEDULED_DEPARTURE|SCHEDULED_TIME|DISTANCE|SCHEDULED_ARRIVAL|DEPARTURE_DELAY|DELAY|
+-----+---+-----------+-------+--------------+-------------------+--------------+--------+-----------------+---------------+-----+
|    7| 31|          5|      1|             1|               1727|          65.0|     236|             1832|           -4.0|    0|
|    6|  5|          5|      2|             2|                720|         104.0|     404|              904|            0.0|    0|
|    7| 14|          2|      1|             3|               1245|         133.0|     726|             1458|            0.0|    0|
|    5| 26|          2|      3|             4|               1140|         125.0|     669|             1445|           -3.0|    0|
|    1|  9|          5|      1|             5|               1040|          47.0|  

In [26]:
full_df.write.parquet("/mnt/data/full_df.parquet")

In [ ]:
flight_df = sqlCtx.createDataFrame(flights_new_bal)
flight_df.show(5)

In [ ]:
flight_df.write.parquet("/mnt/data/flight_df.parquet")

In [29]:
flightsParquetSample = flights_new_bal.sample(n=1000, random_state=314)

In [30]:
small_df = sqlCtx.createDataFrame(flightsParquetSample)
small_df.show(5)

+-----+---+-----------+-------+--------------+-------------------+--------------+--------+-----------------+---------------+-----+
|MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|SCHEDULED_DEPARTURE|SCHEDULED_TIME|DISTANCE|SCHEDULED_ARRIVAL|DEPARTURE_DELAY|DELAY|
+-----+---+-----------+-------+--------------+-------------------+--------------+--------+-----------------+---------------+-----+
|    6| 23|          2|      2|            97|                800|         172.0|    1080|             1052|           -6.0|    0|
|    7| 29|          3|      4|             4|                900|         160.0|    1009|              940|            5.0|    1|
|    7| 17|          5|      2|            10|               2120|          85.0|     352|             2145|           14.0|    1|
|    5| 13|          3|      8|             9|               1419|         118.0|     596|             1617|           -2.0|    0|
|    4| 13|          1|      5|            17|               1705|          50.0|  

In [31]:
small_df.write.parquet("/mnt/data/small_df.parquet")